# Line Detection V1
1. เวอร์ชั่นนี้จะเป็นการตรวจจับบรรทัดแบบทั้งแถวเลย
2. เรียงตัวอักษรจากซ้ายไปขวา

## Histogram Projection
ใช้ Histogram Projection เพื่อแยกบรรทัดในแนวนอน:

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

output_folder = Path("../data/output_images/output_lines")
output_folder.mkdir(exist_ok=True)

# โหลดภาพ
image = cv2.imread("../data/test_images/line_detection/testAtoZ_1.jpg")
if image is None:
    raise FileNotFoundError("ไม่พบไฟล์ภาพ กรุณาตรวจสอบเส้นทางของไฟล์")

# แปลงเป็น Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

'''
# แปลงเป็น Binary
_, binary_image = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)  # Threshold ขาวดำ
'''

# การ Threshold แบบ Adaptive Gaussian
binary_image = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51,30
)

# คำนวณ Histogram Projection
horizontal_projection = np.sum(binary_image, axis=1)  # รวมพิกเซลในแนวนอน

# หาตำแหน่งที่มีข้อความ
line_indices = np.where(horizontal_projection > 0)[0]

# แยกช่วงตำแหน่งแต่ละบรรทัด
lines = []
start_idx = None

for i in range(len(line_indices) - 1):
    if start_idx is None:
        start_idx = line_indices[i]
    if line_indices[i+1] > line_indices[i] + 1:  # เจอช่วงว่างระหว่างบรรทัด
        end_idx = line_indices[i]
        lines.append((start_idx, end_idx))
        start_idx = None
if start_idx is not None:
    lines.append((start_idx, line_indices[-1]))

print("Lines detected:", lines)

# แยกบรรทัดและเพิ่ม Padding
line_images = []
padding = 5  # ระบุ Padding

for start, end in lines:
    padded_start = max(0, start - padding)
    padded_end = min(binary_image.shape[0], end + padding)
    line_img = binary_image[padded_start:padded_end, :]
    line_images.append(line_img)

for idx, line_img in enumerate(line_images):
    file_path = output_folder / f"line_{idx + 1}.png"
    cv2.imwrite(str(file_path), line_img)
print(f"Saved lines to {output_folder}")

'''
# เพิ่มเส้นกรอบรอบข้อความแต่ละบรรทัด
for start, end in lines:
    cv2.rectangle(image, (0, start), (image.shape[1], end), (0, 255, 0), 2)
cv2.imshow("Detected Lines", image)
cv2.imshow("Binary Image", binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# การแยกข้อความแต่ละบรรทัดออกเป็นภาพย่อย
line_images = [binary_image[start:end, :] for start, end in lines]
for idx, line_img in enumerate(line_images):
    cv2.imshow(f"Line {idx + 1}", line_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''


Lines detected: [(36, 61)]
Saved lines to ..\data\output_images\output_lines


'\n# เพิ่มเส้นกรอบรอบข้อความแต่ละบรรทัด\nfor start, end in lines:\n    cv2.rectangle(image, (0, start), (image.shape[1], end), (0, 255, 0), 2)\ncv2.imshow("Detected Lines", image)\ncv2.imshow("Binary Image", binary_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n\n# การแยกข้อความแต่ละบรรทัดออกเป็นภาพย่อย\nline_images = [binary_image[start:end, :] for start, end in lines]\nfor idx, line_img in enumerate(line_images):\n    cv2.imshow(f"Line {idx + 1}", line_img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

## Word/Character Detection
ใช้ cv2.findContours เพื่อตรวจจับตัวอักษรหรือคำในแต่ละบรรทัด:

### แบบใหม่

In [9]:
import cv2
import numpy as np
from pathlib import Path

# สร้างโฟลเดอร์สำหรับเก็บตัวอักษร
output_folder = Path("../data/output_images/output_characters")  # ใช้ Path จาก pathlib
output_folder.mkdir(exist_ok=True)  # สร้างโฟลเดอร์ หากไม่มีอยู่แล้ว

char_images = []

for idx, line_img in enumerate(line_images):
    # หา Contours ในภาพแต่ละบรรทัด
    contours, _ = cv2.findContours(line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # สร้างสำเนาภาพเพื่อวาดกรอบ
    line_with_boxes = cv2.cvtColor(line_img, cv2.COLOR_GRAY2BGR)
    
    # เรียงตัวอักษรตามตำแหน่งซ้ายไปขวา
    sorted_contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    
    for contour in sorted_contours:
        # คำนวณกรอบสี่เหลี่ยมรอบ Contour
        x, y, w, h = cv2.boundingRect(contour)

        # เพิ่มขนาดกรอบ 10%
        padding_x = int(w * 0.1)  # เพิ่ม 10% ของความกว้าง
        padding_y = int(h * 0.1)  # เพิ่ม 10% ของความสูง

        # ปรับขอบเขตของกรอบ
        x = max(0, x - padding_x)
        y = max(0, y - padding_y)
        w = min(line_img.shape[1] - x, w + 2 * padding_x)
        h = min(line_img.shape[0] - y, h + 2 * padding_y)
        
        # กำหนดเงื่อนไขเพื่อกรอง Noise หรือกล่องเล็กเกินไป
        if 2 < w < 100 and 2 < h < 100:  # ปรับค่าขนาดขั้นต่ำและสูงสุดตามต้องการ
            # ตัดภาพตัวอักษร
            char_img = line_img[y:y+h, x:x+w]
            char_images.append(char_img)

            # บันทึกตัวอักษร
            file_path = output_folder / f"line_{idx + 1}_char_{len(char_images)}.png"
            cv2.imwrite(str(file_path), char_img)  # แปลง Path เป็น String เพื่อใช้กับ OpenCV

            # วาดกรอบสี่เหลี่ยมในภาพต้นฉบับ
            cv2.rectangle(line_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # บันทึกภาพพร้อมกรอบตัวอักษร (ถ้าต้องการ)
    boxed_line_path = output_folder / f"line_{idx + 1}_boxed.png"
    cv2.imwrite(str(boxed_line_path), line_with_boxes)

    print(f"Line {idx + 1}: Detected {len(sorted_contours)} characters")

# รายงานผลลัพธ์
print(f"Saved {len(char_images)} characters to {output_folder}")


Line 1: Detected 26 characters
Saved 26 characters to ..\data\output_images\output_characters


## Predict images

In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# ฟังก์ชันเพิ่ม Padding เพื่อปรับภาพเป็นขนาดสี่เหลี่ยมจัตุรัส 32x32
def pad_to_square(image, desired_size):
    old_size = image.shape[:2]  # (height, width)
    delta_w = desired_size - old_size[1]  # ส่วนที่ต้องเพิ่มในความกว้าง
    delta_h = desired_size - old_size[0]  # ส่วนที่ต้องเพิ่มในความสูง
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    # เพิ่ม Padding รอบภาพ
    color = [0]  # สีดำสำหรับ Grayscale
    padded_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return padded_image

# โหลดโมเดลที่ฝึกไว้
model_path = "../models/eng_uppercase_model.h5"
model = load_model(model_path)

# กำหนดขนาด Input ของโมเดล
input_size = 32  # ขนาด 32x32

# เตรียมข้อมูลจาก char_images
processed_images = []

for char_img in char_images:
    # เพิ่ม Padding และปรับขนาดภาพ
    padded_img = pad_to_square(char_img, input_size)
    
    # Normalization (เปลี่ยนค่าพิกเซลให้อยู่ในช่วง [0, 1])
    normalized_img = padded_img / 255.0
    
    # เพิ่มช่องทางสี (Channel) สำหรับโมเดลที่รองรับ Input แบบ (Height, Width, Channels)
    if len(normalized_img.shape) == 2:  # หากภาพเป็น Grayscale (2D)
        normalized_img = np.expand_dims(normalized_img, axis=-1)
    
    # เก็บภาพที่ประมวลผลแล้ว
    processed_images.append(normalized_img)

# แปลงเป็น NumPy Array
processed_images = np.array(processed_images)

# ทำนายผลลัพธ์
predictions = model.predict(processed_images)

# แปลงค่าความน่าจะเป็นเป็นคลาสที่คาดการณ์ได้
predicted_classes = np.argmax(predictions, axis=1)

# แสดงผลลัพธ์พร้อมภาพ
for idx, char_img in enumerate(char_images):
    predicted_label = predicted_classes[idx]
    print(f"Character {idx + 1}: Predicted Label = {predicted_label}")
    #cv2.imshow(f"Character {idx + 1}", char_img)

#cv2.waitKey(0)
#cv2.destroyAllWindows()


1/1 [==============================] - 0s 75ms/step
Character 1: Predicted Label = 0
Character 2: Predicted Label = 1
Character 3: Predicted Label = 2
Character 4: Predicted Label = 3
Character 5: Predicted Label = 4
Character 6: Predicted Label = 5
Character 7: Predicted Label = 6
Character 8: Predicted Label = 7
Character 9: Predicted Label = 8
Character 10: Predicted Label = 9
Character 11: Predicted Label = 10
Character 12: Predicted Label = 11
Character 13: Predicted Label = 12
Character 14: Predicted Label = 13
Character 15: Predicted Label = 14
Character 16: Predicted Label = 15
Character 17: Predicted Label = 16
Character 18: Predicted Label = 17
Character 19: Predicted Label = 18
Character 20: Predicted Label = 19
Character 21: Predicted Label = 20
Character 22: Predicted Label = 21
Character 23: Predicted Label = 22
Character 24: Predicted Label = 23
Character 25: Predicted Label = 24
Character 26: Predicted Label = 25
